In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image

In [4]:
# Define the root directories for your train and test datasets
train_data_dir = 'C:/Users/Patrik/Desktop/Files/fei/Ing/2nd class/HNS/project/Dataset_Znacky/Train-adj-train-test/train'
test_data_dir = 'C:/Users/Patrik/Desktop/Files/fei/Ing/2nd class/HNS/project/Dataset_Znacky/Train-adj-train-test/test'

# Create a list of unique class labels from the train dataset
unique_labels = sorted(os.listdir(train_data_dir))

# Create a mapping from class labels to sequential integers
label_to_int = {label: i for i, label in enumerate(unique_labels)}

# Define transforms for data augmentation and normalization
data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(299),  # InceptionV3's input size is 299x299
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data_dir, label_to_int, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.samples = []

        for class_name in os.listdir(data_dir):
            class_dir = os.path.join(data_dir, class_name)

            if os.path.isdir(class_dir):
                images = os.listdir(class_dir)
                for img in images:
                    image_path = os.path.join(class_dir, img)
                    label = label_to_int[class_name]
                    self.samples.append((image_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image, label


In [5]:
# Create custom datasets for train and test
train_dataset = CustomDataset(train_data_dir, label_to_int, data_transforms)
test_dataset = CustomDataset(test_data_dir, label_to_int, data_transforms)

# Create data loaders for training and testing
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Load a pretrained InceptionV3 model
model = models.inception_v3(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(unique_labels))  # Output units match the number of unique labels



c:\Users\Patrik\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Patrik\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Set the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
device = torch.device("cpu")  # Use "cuda" if available
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs, aux_outputs = model(inputs)  # InceptionV3 has an auxiliary output
        loss1 = criterion(outputs, labels)
        loss2 = criterion(aux_outputs, labels)
        loss = loss1 + 0.4 * loss2  # Combine the two losses

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")



KeyboardInterrupt: 

In [7]:
# Test the model on the testing data
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

KeyboardInterrupt: 